# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../data/')

In [2]:
import pandas as pd
import knapsack
import cvxpy as cp
import numpy as np
import requests

# Toy 0-1 one-dimensional knapsack example

In [ ]:
o_values = np.array([22, 12, 16, 10, 35, 26, 42, 53])
o_weights = np.array([21, 11, 15, 9, 34, 25, 41, 52])
o_capacity = 100

## `knapsack`

In [ ]:
# Maximize sum of selected value. Sum of selected weight is less than capacity

total_value, items_idx = knapsack.knapsack(o_weights, o_values).solve(o_capacity)
items = [o_values[i] for i in items_idx]

print(sum(items))
print(items)

## `cvxpy`

In [ ]:
o_x = cp.Variable(8, boolean=True)

In [ ]:
o_prob = cp.Problem(
    cp.Maximize(o_values@o_x),
    [o_weights@o_x <= o_capacity]
)

In [ ]:
o_prob.solve()

In [ ]:
o_selection = [int(round(j)) for j in o_x.value]
o_selection

# Toy 0-1 multidimensional knapsack example

In [ ]:
m_values = np.array([22, 12, 16, 10, 35, 26, 42, 53])
m_weights = np.array([
    [21, 11, 15, 9, 34, 25, 41, 52],
    [28, 8, 25, 3, 34, 25, 35, 60]
])
m_capacity = np.array([100, 125])
m_num = 3

In [ ]:
m_x = cp.Variable(8, boolean=True)

In [ ]:
m_prob = cp.Problem(
    cp.Maximize(m_values@m_x),
    [
        m_weights@m_x <= m_capacity,
        np.ones(8)@m_x == m_num
    ]
)

In [ ]:
m_prob.solve()

In [ ]:
m_selection = [int(round(j)) for j in m_x.value]
m_selection

# Toy team optimisation

In [3]:
bootstrap_static_request = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
bootstrap_static_d = bootstrap_static_request.json()

In [4]:
black_list_players = ['Pogba']
players = [i for i in bootstrap_static_d['elements'] if i['web_name'] not in black_list_players]

In [5]:
player_names = np.array([i['web_name'] for i in players])
player_points = np.array([i['total_points'] for i in players])/38
player_costs = np.array([[i['now_cost'] for i in players]])
player_position = np.array([i['element_type'] for i in players])
player_team = np.array([i['team'] for i in players])

In [6]:
player_position_weights = np.zeros((4, len(players)))
for i in range(0, 4):
    for j in range(0, len(players)):
        if player_position[j] == i+1:
            player_position_weights[i, j] = 1
        else:
            player_position_weights[i, j] = 0

In [7]:
player_team_weights = np.zeros((20, len(players)))
for i in range(0, 20):
    for j in range(0, len(players)):
        if player_team[j] == i+1:
            player_team_weights[i, j] = 1
        else:
            player_team_weights[i, j] = 0

In [8]:
player_position_capacity = [1, 4, 5, 1]
bench_position_capacity = [1, 1, 0, 2]

In [9]:
min_cost_keeper = np.min([i['now_cost'] for i in players if i['element_type'] == 1])
min_cost_defender = np.min([i['now_cost'] for i in players if i['element_type'] == 2])
min_cost_midfielder = np.min([i['now_cost'] for i in players if i['element_type'] == 3])
min_cost_striker = np.min([i['now_cost'] for i in players if i['element_type'] == 4])
min_cost_bench = \
bench_position_capacity@np.array([min_cost_keeper, min_cost_defender, min_cost_midfielder, min_cost_striker])

In [10]:
player_cost_capacity = [1000 - min_cost_bench]
player_team_capacity = [3]*20

In [11]:
player_weights = np.concatenate((
    player_costs,
    player_position_weights,
    player_team_weights
), axis=0)
player_capacity = np.array(
    player_cost_capacity
    + player_position_capacity
    + player_team_capacity
)

player_num = 11

In [12]:
bench_num = 4

## From scratch

In [13]:
fs_player_x = cp.Variable(len(players), boolean=True)

In [14]:
fs_player_prob = cp.Problem(
    cp.Maximize(player_points@fs_player_x),
    [
        player_weights@fs_player_x <= player_capacity,
        np.ones(len(players))@fs_player_x == player_num
    ]
)

In [15]:
fs_player_prob.solve()

54.21052630930314

In [16]:
fs_player_selection = [int(round(j)) for j in fs_player_x.value]
[i for i, j in zip(player_names, fs_player_selection) if j == 1]

['Fraser',
 'David Luiz',
 'Kanté',
 'Milivojevic',
 'Pickford',
 'Robertson',
 'van Dijk',
 'Salah',
 'Laporte',
 'Sterling',
 'Jiménez']

In [17]:
fs_bench_team_capacity = player_team_capacity - player_team_weights@fs_player_selection

In [18]:
fs_bench_capacity = np.array(
    [min_cost_bench]
    + bench_position_capacity
    + list(fs_bench_team_capacity)
)

In [19]:
fs_bench_x = cp.Variable(len(players), boolean=True)

In [20]:
fs_bench_prob = cp.Problem(
    cp.Maximize(player_points@fs_bench_x),
    [
        player_weights@fs_bench_x <= fs_bench_capacity,
        np.ones(len(players))@fs_bench_x == bench_num
    ]
)

In [21]:
fs_bench_prob.solve()

1.5789473685256235

In [22]:
fs_bench_selection = [int(round(j)) for j in fs_bench_x.value]
[i for i, j in zip(player_names, fs_bench_selection) if j == 1]

['Nketiah', 'Button', 'Kelly', 'Wickham']

In [23]:
abs(round(1000 - (player_costs@fs_player_x.value)[0] - min_cost_bench))

0.0

## Existing team

In [45]:
print(list(player_names).index('Salah'))
print(list(player_names).index('Lingard'))

print(list(player_names).index('van Dijk'))
print(list(player_names).index('Stones'))

print(list(player_names).index('Jiménez'))
print(list(player_names).index('Wilson'))

206
258
198
222
436
51


In [46]:
et_x_0 = np.array([abs(round(i)) for i in fs_player_x.value])
et_x_0[206] = 0.0
et_x_0[258] = 1.0
et_x_0[198] = 0.0
et_x_0[222] = 1.0
et_x_0[436] = 0.0
et_x_0[51] = 1.0

In [56]:
et_selection_0 = [int(round(j)) for j in et_x_0]
et_player_selection_0_names = [i for i, j in zip(player_names, et_selection_0) if j == 1]
et_player_selection_0_names

['Wilson',
 'Fraser',
 'David Luiz',
 'Kanté',
 'Milivojevic',
 'Pickford',
 'Robertson',
 'Laporte',
 'Stones',
 'Sterling',
 'Lingard']

In [48]:
et_player_x = cp.Variable(len(players), boolean=True)

In [60]:
et_player_prob = cp.Problem(
    cp.Maximize(
        player_points@et_player_x - 4*(11 - et_x_0@et_player_x)
    ),
    [
        player_weights@et_player_x <= player_capacity,
        np.ones(len(players))@et_player_x == player_num,
#         et_x_0@et_player_x >= 10
    ]
)

In [61]:
et_player_prob.solve()

46.578947302329325

In [55]:
et_player_selection = [int(round(j)) for j in et_player_x.value]
et_player_selection_names = [i for i, j in zip(player_names, et_player_selection) if j == 1]
et_player_selection_names

['Wilson',
 'Fraser',
 'David Luiz',
 'Kanté',
 'Milivojevic',
 'Pickford',
 'Robertson',
 'Salah',
 'Laporte',
 'Stones',
 'Sterling']

In [57]:
print(set(et_player_selection_0_names) - set(et_player_selection_names))
print(set(et_player_selection_names) - set(et_player_selection_0_names))

{'Lingard'}
{'Salah'}
